In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# General imports
import glob
import os
import re
import pickle
import datetime

# Data manipulation
import pandas as pd
import numpy as np

# Basic plotting
import bokeh
from bokeh.io import export_svg
from bokeh.models import NumeralTickFormatter
from bokeh.themes.theme import Theme
theme = Theme(
    json=ga.PLOT_STYLE
)

from bokeh.io import output_notebook
output_notebook()

import holoviews as hv
hv.renderer('bokeh').theme = theme

import panel as pn
pn.config.comms = "vscode"

# Large data plotting
import datashader as ds
from holoviews.operation.datashader import datashade, rasterize

# Making graphs
import networkx as nx
import matplotlib.pyplot as plt
import itertools
from tqdm.auto import tqdm
from multiprocessing import Pool

hv.extension('bokeh')

from SSMuLA import growth_analysis as ga

### Plot hooks

In [ ]:
# Plot Hooks
def one_decimal_x(plot,element):
    plot.handles['plot'].xaxis[0].formatter = NumeralTickFormatter(format="0.0")

def one_decimal_y(plot,element):
    plot.handles['plot'].yaxis[0].formatter = NumeralTickFormatter(format="0.0")

def fixmargins(plot,element):
    plot.handles['plot'].min_border_right=30
    plot.handles['plot'].min_border_left=65
    plot.handles['plot'].min_border_top=20
    plot.handles['plot'].min_border_bottom=65
    plot.handles['plot'].outline_line_color='black'
    plot.handles['plot'].outline_line_alpha=1
    plot.handles['plot'].outline_line_width=1

In [ ]:
# Import the imputed TrpB_data
TrpB_imputed_data = pd.read_csv(
     '../../../data/figure_data/4-site_imputed/20230828_KNN_imputed_TrpB.csv', 
    index_col=0
)

TrpB_imputed_data['imputed'] = True

# Import the measured TrpB_data
TrpB_measured_data = pd.read_csv(
    '../../../data/figure_data/4-site_merged_replicates/20230827/four-site_simplified_AA_data.csv',
)

TrpB_measured_data = TrpB_measured_data[TrpB_measured_data['# Stop'] == 0].copy().drop(columns=['# Stop'])

TrpB_measured_data['imputed'] = False

# Combine and sort the data
TrpB_data = pd.concat([TrpB_imputed_data, TrpB_measured_data]).sort_values('AAs').reset_index(drop=True)

# Add a column where no fitness values are below 0
TrpB_data['fitness (min 0)'] = TrpB_data['fitness'].apply(lambda x: max(0, x))

TrpB_data

In [ ]:
# VDGV is parent

# Import the measured GB1 data
GB1_measured_data = pd.read_csv('../../../data/figure_data/GB1_data/GB1_Fitness.csv').rename(columns={'AAString': 'AAs'}).drop(columns=['Mutations'])

GB1_measured_data['imputed'] = False

# Import the imputed GB1 data
GB1_imputed_data = pd.read_excel('../../../data/figure_data/GB1_data/GB1_missing_data.xlsx').rename(columns={'Variants': 'AAs', 'Imputed fitness': 'Fitness'})

GB1_imputed_data['imputed'] = True

# Combine the data and add AA1 -> AA4 columns
GB1_data = pd.concat([GB1_measured_data, GB1_imputed_data], ignore_index=True).sort_values('AAs').reset_index(drop=True)

for i in range(4):
    GB1_data.insert(i+1, f'AA{i+1}', GB1_data['AAs'].apply(lambda x: x[i]))

# Get the Fitness/max column to scale the data the same way as the TrpB data
GB1_data['Fitness/max'] = GB1_data['Fitness'] / GB1_data['Fitness'].max()
GB1_fit_min = 0.01

# Only set as active if they are not imputed and have a fitness above the minimum. This will prevent them from being included as starting points in the path analysis, but they will still appear in the graphs.
GB1_data['active'] = GB1_data.apply(lambda x: (x['Fitness'] > GB1_fit_min) & (x['imputed'] == False), axis=1)

GB1_data.sort_values('Fitness', ascending=False)

### Importantly, note that the GB1 imputation resulted in the top variant being one of the imputed ones, which is not ideal for downstream analyses. 
Here, we treat it as the maximum variant, but we do not count any of the imputed variants as active variants when deciding what we consider as starting points.

# Simulate different experimental approaches to directed evolution

I want to be able to take in a DataFrame of sequence and fitness (already has stop codons removed) as well as a "zero-fitness" cutoff and either simulate (random samples) or exhaustively calculate all the possible campaigns. It will likely be better to write functions to exhaustively calculate that can then be converted to simulateions.

#### Inputs
- DataFrame of sequence and fitness (without stop codons!)
    - sequence column name
    - fitness column name
- zero-fitness cutoff
- number of samples to simulate

#### Outputs
- DataFrame
    - start sequence
    - end sequence
    - start fitness
    - end fitness
    - optional: order of steps taken (order that positions were targeted)

In [ ]:
def print_characteristics(df):
    
    print('mean:',np.mean(df['final_fitness'].values))
    print('median:',np.median(df['final_fitness'].values))
    print('fraction reaching max:', sum(df['final_fitness'].values == 1) / len(df['final_fitness'].values))

In [ ]:
def run_all_simulations(
    df, 
    seq_col, 
    fitness_col, 
    n_sites=4, 
    N=96, 
    max_samples=None,
    n_jobs=16
):

    ######## Simulate a single step directed evolution walk ########
    print(f'Simulate a single step directed evolution walk')
    fitness_array,single_step_DE = ga.simulate_single_step_DE(
        data=df, 
        seq_col=seq_col, 
        fitness_col=fitness_col,
        n_sites=n_sites,
    )
    print_characteristics(single_step_DE)

    ######## Simulate a simple SSM recombination ########
    print('\nSimulate a simple SSM recombination')
    SSM_recomb = ga.simulate_simple_SSM_recomb_DE(
        data=df, 
        seq_col=seq_col, 
        fitness_col=fitness_col, 
        n_sites=n_sites,
    )
    print_characteristics(SSM_recomb)

    ######## Simulate SSM predict top N ########
    print(f'\nSimulate SSM predict top {N}')
    SSM_pred96 = ga.sample_SSM_test_top_N(
        data=df, 
        seq_col=seq_col, 
        fitness_col=fitness_col,
        n_sites=n_sites,
        N=N,
        max_samples=max_samples,
        n_jobs=n_jobs,
    )
    print_characteristics(SSM_pred96)

    return {'SSM predict top 96':SSM_pred96, 'single step SSM':single_step_DE, 'SSM recomb':SSM_recomb}

In [ ]:
all_active_results = {}

print('\nRunning TrpB simulations:\n')
all_active_results['TrpB'] = run_all_simulations(
    TrpB_data,
    'AAs',
    'fitness (min 0)',
    n_sites=4,
    N=96,
    max_samples=None,
    n_jobs=16
)

######## GB1 INPUTS ########

print('\nRunning GB1 simulations:\n')
all_active_results['GB1'] = run_all_simulations(
    GB1_data,
    'AAs',
    'Fitness/max',
    n_sites=4,
    N=96,
    max_samples=None,
    n_jobs=16
)

## Make a violin plot of the fitness distribution of the final sequence. Group by the type of simulation and split by the protein.

In [ ]:
dfs = []
for label,df in all_active_results['TrpB'].items():
    df['simulation'] = label
    df['protein'] = 'TrpB'
    dfs.append(df)

for label,df in all_active_results['GB1'].items():
    df['simulation'] = label
    df['protein'] = 'GB1'
    dfs.append(df)

all_active_df = pd.concat(dfs).reset_index(drop=True)
all_active_df['final_fitness'] = all_active_df['final_fitness'].astype(float)
all_active_df

In [ ]:
hv.extension('bokeh')

In [ ]:
violin = hv.Violin(
        all_active_df,
        kdims=['simulation', 'protein'],
        vdims=['final_fitness'],
    ).opts(
        split='protein',
        frame_width=400, 
        frame_height=300,
        fontscale=1.25,
        legend_position='top',
        show_legend=True,
        inner=None,
        violin_width=1,
        title='all active starts w/ no imputed',
        hooks=[fixmargins, one_decimal_y],
        ylabel = 'max fitness achieved'
    )
ECDF = hv.Curve(
    all_active_df.sort_values(['simulation', 'protein', 'final_fitness']),
    kdims = 'final_fitness',
    vdims = ['final_fitness ECDF', 'protein', 'simulation'],
).groupby(
    ['simulation', 'protein']
).opts(color=hv.Cycle('Category20')).overlay().opts(
    frame_width=450,
    frame_height=300,
    fontscale=1.25,
    legend_position = 'right',
    title='all active starts w/ no imputed',
    hooks=[fixmargins, one_decimal_x, one_decimal_y],
    xlabel = 'max fitness achieved',
    ylabel = 'ECDF'
)

violin + ECDF

### What does it look like if I enforce starting from the top 9783 variants of GB1 as well as the top 9783 variants of TrpB?

In [ ]:
top9783_results = all_active_results.copy()
temp_GB1 = GB1_data.copy()

GB1_fit_min = GB1_data[GB1_data['active']].sort_values('Fitness/max', ascending=False).reset_index(drop=True).iloc[9782]['Fitness']

temp_GB1['active'] = temp_GB1.apply(lambda x: (x['Fitness'] >= GB1_fit_min) & (x['imputed'] == False), axis=1)
######## GB1 INPUTS ########

print('\nRunning GB1 simulations:\n')
top9783_results['GB1'] = run_all_simulations(
    temp_GB1,
    'AAs',
    'Fitness/max',
    n_sites=4,
    N=96,
    max_samples=None,
    n_jobs=16
)

In [ ]:
dfs = []
for label,df in top9783_results['TrpB'].items():
    df['simulation'] = label
    df['protein'] = 'TrpB'
    dfs.append(df)

for label,df in top9783_results['GB1'].items():
    df['simulation'] = label
    df['protein'] = 'GB1'
    dfs.append(df)

top9783_df = pd.concat(dfs).reset_index(drop=True)
top9783_df['final_fitness'] = top9783_df['final_fitness'].astype(float)

violin_top9783 = hv.Violin(
        top9783_df,
        kdims=['simulation', 'protein'],
        vdims=['final_fitness'],
    ).opts(
        split='protein',
        frame_width=400, 
        frame_height=300,
        fontscale=1.25,
        legend_position='top',
        show_legend=True,
        inner=None,
        violin_width=1,
        title='top 9783 active starts w/ no imputed',
        hooks=[fixmargins, one_decimal_y],
        ylabel = 'max fitness achieved'
    )

violin + violin_top9783

In [ ]:
ECDF_top9783 = hv.Curve(
    top9783_df.sort_values(['simulation', 'protein', 'final_fitness']),
    kdims = 'final_fitness',
    vdims = ['final_fitness ECDF', 'protein', 'simulation'],
).groupby(
    ['simulation', 'protein']
).opts(color=hv.Cycle('Category20')).overlay().opts(
    frame_width=450,
    frame_height=300,
    fontscale=1.25,
    legend_position = 'right',
    title='top 9783 active starts w/ no imputed',
    hooks=[fixmargins, one_decimal_x, one_decimal_y],
    xlabel = 'max fitness achieved',
    ylabel = 'ECDF'
)

ECDF + ECDF_top9783

### What does it look like if I set the same fraction of max?

In [ ]:
top_min_frac_results = all_active_results.copy()
temp_GB1 = GB1_data.copy()

GB1_fit_min = TrpB_data[TrpB_data['active']]['fitness'].min()

temp_GB1['active'] = temp_GB1.apply(lambda x: (x['Fitness/max'] >= GB1_fit_min) & (x['imputed'] == False), axis=1)
######## GB1 INPUTS ########

print('\nRunning GB1 simulations:\n')
top_min_frac_results['GB1'] = run_all_simulations(
    temp_GB1,
    'AAs',
    'Fitness/max',
    n_sites=4,
    N=96,
    max_samples=None,
    n_jobs=16
)

In [ ]:
dfs = []
for label,df in top_min_frac_results['TrpB'].items():
    df['simulation'] = label
    df['protein'] = 'TrpB'
    dfs.append(df)

for label,df in top_min_frac_results['GB1'].items():
    df['simulation'] = label
    df['protein'] = 'GB1'
    dfs.append(df)

top_min_frac_df = pd.concat(dfs).reset_index(drop=True)
top_min_frac_df['final_fitness'] = top_min_frac_df['final_fitness'].astype(float)

violin_top_min_frac = hv.Violin(
        top_min_frac_df,
        kdims=['simulation', 'protein'],
        vdims=['final_fitness'],
    ).opts(
        split='protein',
        frame_width=400, 
        frame_height=300,
        fontscale=1.25,
        legend_position='top',
        show_legend=True,
        inner=None,
        violin_width=1,
        title='same fraction active starts w/ no imputed',
        hooks=[fixmargins, one_decimal_y],
        ylabel = 'max fitness achieved'
    )
violin + violin_top9783 + violin_top_min_frac

In [ ]:
ECDF_top_min_frac = hv.Curve(
    top_min_frac_df.sort_values(['simulation', 'protein', 'final_fitness']),
    kdims = 'final_fitness',
    vdims = ['final_fitness ECDF', 'protein', 'simulation'],
).groupby(
    ['simulation', 'protein']
).opts(color=hv.Cycle('Category20')).overlay().opts(
    frame_width=450,
    frame_height=300,
    fontscale=1.25,
    legend_position = 'right',
    title='same fraction active starts w/ no imputed',
    hooks=[fixmargins, one_decimal_x, one_decimal_y],
    xlabel = 'max fitness achieved',
    ylabel = 'ECDF'
)

ECDF + ECDF_top9783 + ECDF_top_min_frac

## Export figures

In [ ]:
figure_2c = violin_top9783.opts(
    hooks=[fixmargins, one_decimal_y],
    ylabel = 'max fitness achieved',
)

figure_2d = ECDF_top9783.opts(
    hooks=[fixmargins, one_decimal_x, one_decimal_y],
    xlabel = 'max fitness achieved',
    ylabel = 'ECDF'
)

figure_2c + figure_2d

In [ ]:
hv.Violin(
        top9783_df,
        kdims=['simulation', 'protein'],
        vdims=['final_fitness'],
    ).opts(
        split='protein',
        frame_width=400, 
        frame_height=300,
        fontscale=1.25,
        legend_position='top',
        show_legend=True,
        inner=None,
        violin_width=1,
        title='top 9783 active starts w/ no imputed',
        hooks=[fixmargins, one_decimal_y],
        ylabel = 'max fitness achieved',
)+\
hv.Curve(
    top9783_df.sort_values(['simulation', 'protein', 'final_fitness']),
    kdims = 'final_fitness',
    vdims = ['final_fitness ECDF', 'protein', 'simulation'],
).groupby(
    ['simulation', 'protein']
).opts(color=hv.Cycle('Category20')).overlay().opts(
    frame_width=450,
    frame_height=300,
    fontscale=1.25,
    legend_position = 'right',
    title='top 9783 active starts w/ no imputed',
    hooks=[fixmargins, one_decimal_x, one_decimal_y],
    xlabel = 'max fitness achieved',
    ylabel = 'ECDF'
)

In [ ]:
# plot=hv.render(figure_2c, backend='bokeh')
# plot.output_backend = "svg"

# filename='figure3b_DE_violins.svg'
# export_svg(plot, filename=filename)

# plot=hv.render(figure_2d, backend='bokeh')
# plot.output_backend = "svg"

# filename='figure3c_DE_ECDFs.svg'
# export_svg(plot, filename=filename)

### What if we test the top 196 or 384?

In [ ]:
### TrpB simulations ###
compare_N_dict = {
    'TrpB':{}, 
    'GB1':{}
}

print('\nRunning TrpB simulations:\n')

for N in [96,192,384]:

    print(f'\nSimulate SSM predict top {N}')

    compare_N_dict['TrpB'][N] = ga.sample_SSM_test_top_N(
        data=TrpB_data, 
        seq_col='AAs', 
        fitness_col='fitness (min 0)',
        n_sites=4,
        N=N,
        max_samples=None,
        n_jobs=16,
    )
    print_characteristics(compare_N_dict['TrpB'][N])


### GB1 Simulations ###
print('\nRunning GB1 simulations:\n')
temp_GB1_data = GB1_data.copy()

GB1_fit_min = GB1_data[GB1_data['active']].sort_values('Fitness/max', ascending=False).reset_index(drop=True).iloc[9782]['Fitness']

temp_GB1_data['active'] = temp_GB1_data.apply(lambda x: (x['Fitness'] >= GB1_fit_min) & (x['imputed'] == False), axis=1)

for N in [96,192,384]:

    print(f'\nSimulate SSM predict top {N}')

    compare_N_dict['GB1'][N] = ga.sample_SSM_test_top_N(
        data=temp_GB1_data, 
        seq_col='AAs', 
        fitness_col='Fitness/max',
        n_sites=4,
        N=N,
        max_samples=None,
        n_jobs=16,
    )
    print_characteristics(compare_N_dict['GB1'][N])

In [ ]:
dfs = []
for protein,protein_dict in compare_N_dict.items():
    for N,df in protein_dict.items():
        df['protein'] = protein
        df['N'] = N
        dfs.append(df)

compare_N_df = pd.concat(dfs).reset_index(drop=True)
compare_N_df['final_fitness'] = compare_N_df['final_fitness'].astype(float)

violin_compare_N = hv.Violin(
        compare_N_df,
        kdims=['N', 'protein'],
        vdims=['final_fitness'],
    ).opts(
        split='protein',
        frame_width=400, 
        frame_height=300,
        fontscale=1.25,
        legend_position='right',
        show_legend=True,
        inner=None,
        violin_width=1,
        title='top 9783 active starts SSM predict top N'
    )

ECDF_compare_N = hv.Curve(
    compare_N_df.sort_values(['N', 'protein', 'final_fitness']),
    kdims = 'final_fitness',
    vdims = ['final_fitness ECDF', 'protein', 'N'],
).groupby(
    ['N', 'protein']
).opts(color=hv.Cycle('Category20')).overlay().opts(
    frame_width=400,
    frame_height=300,
    fontscale=1.25,
    legend_position = 'right',
    title='top 9783 active starts SSM predict top N'
)

violin_compare_N + ECDF_compare_N

### What if we start from any variant with fitness > 0?
If 0 is allowed then the additive calculation begins to error.

In [ ]:
all_starts_results = {}

####### TrpB Inputs #######

temp_TrpB_data = TrpB_data.copy()
temp_TrpB_data['active'] = temp_TrpB_data['fitness (min 0)'] > 0

print('\nRunning TrpB simulations:\n')
all_starts_results['TrpB'] = run_all_simulations(
    temp_TrpB_data,
    'AAs',
    'fitness (min 0)',
    n_sites=4,
    N=96,
    max_samples=None,
    n_jobs=16
)

In [ ]:
######## GB1 INPUTS ########

temp_GB1_data = GB1_data.copy()
temp_GB1_data['active'] = temp_GB1_data['Fitness/max'] > 0

print('\nRunning GB1 simulations:\n')
all_starts_results['GB1'] = run_all_simulations(
    temp_GB1_data,
    'AAs',
    'Fitness/max',
    n_sites=4,
    N=96,
    max_samples=None,
    n_jobs=16
)

In [ ]:
dfs = []
for label,df in all_starts_results['TrpB'].items():
    df['simulation'] = label
    df['protein'] = 'TrpB'
    dfs.append(df)

for label,df in all_starts_results['GB1'].items():
    df['simulation'] = label
    df['protein'] = 'GB1'
    dfs.append(df)

all_starts_df = pd.concat(dfs).reset_index(drop=True)
all_starts_df['final_fitness'] = all_starts_df['final_fitness'].astype(float)

violin_all_starts = hv.Violin(
        all_starts_df,
        kdims=['simulation', 'protein'],
        vdims=['final_fitness'],
    ).opts(
        split='protein',
        frame_width=400, 
        frame_height=300,
        fontscale=1.25,
        legend_position='top',
        show_legend=True,
        inner=None,
        violin_width=1,
        title='all starts'
    )
ECDF_all_starts = hv.Curve(
    all_starts_df.sort_values(['simulation', 'protein', 'final_fitness']),
    kdims = 'final_fitness',
    vdims = ['final_fitness ECDF', 'protein', 'simulation'],
).groupby(
    ['simulation', 'protein']
).opts(color=hv.Cycle('Category20')).overlay().opts(
    frame_width=400,
    frame_height=300,
    fontscale=1.25,
    legend_position = 'right',
    title='all starts'
)

violin_all_starts + ECDF_all_starts

### Export notebook as HTML

In [ ]:
# os.system('jupyter nbconvert --to html DE_simulations.ipynb')